# Dog Anxiety Detection Algorithm using Computer Vision
Author: Huidong Hou
The goal of this network is designed, to detect dog anxiety level, using knowledge in computer vision. 
- Note the above was the general goal, but the current version is to detect whether the dog is anxious or not, like a binary classification
- The further goal, is to design a probablistic output (probability that the dog is anxious and probability that it is not), if I understand this correctly

## Table of Contents
- [1 - Packages](#1)
- [2 - Load the Data Set](#2)
- [3 - Crafting the Neural Networks](#3)

<a name = '1'></a>
# 1 - Data Preparation 
 The following import statements are served to bring in all packages, that will be used by the model




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import ToTensor
from torchvision import datasets
from torchvision.transforms import ToTensor
import cv2
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from natsort import natsorted
from torchvision import datasets
import torchvision
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

## I: Imported Package and Setup
Copying the Source code from github, before we use the dataset class to create the data

In [ ]:
!rm -r BC-Dog-Anxiety-Detection-Project
!git clone https://github.com/yangcheng99/BC-Dog-Anxiety-Detection-Project.git


rm: cannot remove 'BC-Dog-Anxiety-Detection-Project': No such file or directory
Cloning into 'BC-Dog-Anxiety-Detection-Project'...
remote: Enumerating objects: 1813, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 1813 (delta 88), reused 176 (delta 86), pack-reused 1632
Receiving objects: 100% (1813/1813), 365.89 MiB | 34.99 MiB/s, done.
Resolving deltas: 100% (388/388), done.
Checking out files: 100% (938/938), done.


Now I will create the Custom Dataset, using the existed pytorch images. The first is to do with the custom loaddata images

In [ ]:
train_data = 'BC-Dog-Anxiety-Detection-Project/Training/'
test_data = 'BC-Dog-Anxiety-Detection-Project/Testing/'



The following code is more modifying base on the case from this [website](https://medium.com/analytics-vidhya/creating-a-custom-dataset-and-dataloader-in-pytorch-76f210a1df5d), in case you want to learn more about creating the dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,path,transforms = None):
		   self.imgs_path = path
		   file_list = glob.glob(self.imgs_path + "*")
		   print(file_list)
		   self.data = []
		   for class_path in file_list:
			    class_name = class_path.split("/")[-1]
			    for img_path in glob.glob(class_path + "/*.jpg"):
				    self.data.append([img_path, class_name])
		   self.class_map = {"Normal" : 0, "Anxious": 1}  
		   self.transforms = transforms
       

    def __len__(self):
    # Return the previously computed number of images
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        #print(f"fetch query {idx}: cn {class_name}; path: {img_path}")
        img = cv2.imread(img_path,3) #We are reading a greyscale image, for simplicity
        img = cv2.resize(img,(416,416))
        img = img.transpose([2,1,0])
        class_id = self.class_map[class_name]
        #print(f"class_id = {class_id}")
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.float()
        #print(f"class_id: {class_id}")
        return img_tensor, class_id
  


In [ ]:
from torchvision import transforms
# we here use the transforms for ImageNet challenge
RGB_MEAN = (0.4914, 0.4822, 0.4465)
RGB_STD = (0.2023, 0.1994, 0.2010)
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(RGB_MEAN, RGB_STD),

])

transform_test = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(RGB_MEAN,RGB_STD),

])


Now we will get some sizes of the tensors

The below is some old files

In [ ]:
if __name__ == "__main__":
  train_dataset = CustomDataset(train_data,transforms = transform_train)
  test_dataset = CustomDataset(test_data,transforms = transform_test)
  print(len(train_dataset))
  print(len(test_dataset))

  dog_train_data_loader = DataLoader(train_dataset, batch_size = 4,  shuffle = True)
  dog_test_data_loader = DataLoader(test_dataset, batch_size = 4, shuffle = True)
  print(len(dog_train_data_loader))
  print(len(dog_test_data_loader))

  #print(train_dataset.length())
  


['BC-Dog-Anxiety-Detection-Project/Training/Anxious', 'BC-Dog-Anxiety-Detection-Project/Training/Normal']
['BC-Dog-Anxiety-Detection-Project/Testing/Anxious', 'BC-Dog-Anxiety-Detection-Project/Testing/Normal']
666
243
167
61


This is for debugging purposes, I think therte is still something wrong with the way the dataset is generated, especially on accessing the indices, or notation, which you can see later.

Now we have finished building our custom dataset, now we can actually run the pre-trained machine ResNet(or maybe just simple CNN network, to train the model)

<a name = '2'></a>
# 2 - Model Creation 

## Model Download
Now we will use the ResNet, downloaded it and train it. The idea is based on Prof.Wei's CSCI3343: Computer Vision's Problem Set 5. I have used two different approaches, the handcrafted resnet, and the pre-trained resnet, both of which have some bugs which i can not figure out what happend.

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


## Model Modification
Now we will modify the last  layer, to predict one layer instead. 

In [ ]:
import torch.nn as nn
#### TODO
# Hint: what's the input and output size of the last linear layer
model.fc = nn.Linear(512,1)

torch.cuda.is_available()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Define Loss Function and Optimizer
Now we will define the loss function, but also the optimizer

In [ ]:
import torch.nn.functional as F
def criterion(y_gt, y_pred):
    #### TODO
    # y_gt is between 0-100 -> scale to 0-1
    # y_pred is any real number -> add a sigmoid to squash it to 0-1
    y_gt = y_gt/100
    y_pred = F.sigmoid(y_pred)
    return F.mse_loss(y_pred, y_gt)

import torch.optim as optim

# freeze the weight for all conv layers
# only learn the last linear layer
for name,param in model.named_parameters():
    if 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

#### TODO

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

### Data Training
  This is the training file, we need to build a data transform to take care of the input before we had out to do the prediction

In [ ]:
#### nothing to change in this code block ####

class Config:  
  def __init__(self, **kwargs):
    # util
    self.batch_size = 16
    self.epochs = 0
    self.save_model_path = '' # use your google drive path to save the model
    self.log_interval = 100 # display after number of batches
    self.criterion = F.cross_entropy # loss for classification
    self.mode = 'train'
    for key, value in kwargs.items():
      setattr(self, key, value)
   
class Trainer:  
  def __init__(self, model, config, train_data = None, test_data = None):    
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.epochs = config.epochs
    self.save_model_path = config.save_model_path
    self.log_interval = config.log_interval
    self.mode = config.mode

    self.globaliter = 0
    self.train_loader = None
    self.test_loader = None
    batch_size = config.batch_size
    if self.mode == 'train': # training mode
      self.train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=1)      
      #self.tb = TensorBoardColab()
      self.optimizer = config.optimizer
    
    if test_data is not None: # need evaluation
      self.test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                         shuffle=False, num_workers=1)
    
    self.model = model.to(self.device)
    self.criterion = config.criterion # loss function
    
                
  def train(self, epoch):  
    self.model.train()
    for batch_idx, (data,target) in enumerate(self.train_loader):      
      self.globaliter += 1
      data, target = data.to(self.device), target.to(self.device)
      #print(data.size(), target.size())
      self.optimizer.zero_grad()
      predictions = self.model(data)

      loss = self.criterion(predictions, target)
      loss.backward()
      self.optimizer.step()

      if batch_idx % self.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(self.train_loader.dataset),
                  100. * batch_idx / len(self.train_loader), loss.item()))
        #self.tb.save_value('Train Loss', 'train_loss', self.globaliter, loss.item())
        #self.tb.flush_line('train_loss')
        
        
  def test(self, epoch, do_loss = True, return_pred = False):
    self.model.eval()
    test_loss = 0
    correct = 0
    pred = []
    with torch.no_grad():
      print('Start testing...')
      for data, target in self.test_loader:
        data = data.to(self.device)
        predictions = self.model(data)
        if return_pred:
          pred.append(predictions.detach().cpu().numpy())
        if do_loss:
            target = target.to(self.device)        
            test_loss += self.criterion(predictions, target).item()*len(target)
            prediction = predictions.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()
      if do_loss:
          test_loss /= len(self.test_loader.dataset)
          accuracy = 100. * correct / len(self.test_loader.dataset)
          print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
              test_loss, correct, len(self.test_loader.dataset), accuracy))
      """
      if self.mode == 'train': # add validation data to tensorboard
        self.tb.save_value('Validation Loss', 'val_loss', self.globaliter, test_loss)
        self.tb.flush_line('val_loss')
      """
      if return_pred:
        return np.hstack(pred)
  def main(self):
    pred = []
    if self.mode == 'train':
      for epoch in range(1, self.epochs + 1):          
          self.train(epoch)
          if self.test_loader is not None:
            # exist validation data
            self.test(epoch)
    if (self.save_model_path != ''):
        torch.save(self.model.state_dict(), self.save_model_path)
    elif self.mode == 'test':
      self.test(0)
    elif self.mode == 'deploy':          
      pred = self.test(0, False, True)
      return pred


This code block is a testing code block to see whether a model can be saved into google drive and how, if you want to store it on your local machine, or google drive, you need to change model_path otherwise it will return an error

In [ ]:
from google.colab import drive
model_testing_name = 'testing.pt'
model_path = F"/content/drive/MyDrive/Data/Model/{model_testing_name}"

torch.save(model,model_path)
print(model_testing_name)

testing.pt


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)

# set of hyperparameters that is thrown into the model
train_config = Config(    
    criterion = criterion,
    save_model_path = '', # if you like, use your google drive path to save the model (mount google drive first)
    log_interval = 100, # display after number of batches
    batch_size = 8,
    optimizer = optimizer,
    epochs = 10,
)

mvp_model_path = "mvp.pt"
model_path_mvp = F"/content/drive/MyDrive/Data/Model/{mvp_model_path}"

model_detection = Trainer(model.to(device), train_config, train_data=train_dataset, test_data=test_dataset).main()
torch.save(model_detection,model_path_mvp)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  


Train Epoch: 1 [0/666 (0%)]	Loss: 0.097596
Start testing...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  



Test set: Average loss: 0.0723, Accuracy: 179/243 (74%)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  


Train Epoch: 2 [0/666 (0%)]	Loss: 0.053397
Start testing...

Test set: Average loss: 0.0616, Accuracy: 179/243 (74%)

Train Epoch: 3 [0/666 (0%)]	Loss: 0.045980
Start testing...

Test set: Average loss: 0.0555, Accuracy: 179/243 (74%)

Train Epoch: 4 [0/666 (0%)]	Loss: 0.054345
Start testing...

Test set: Average loss: 0.0501, Accuracy: 179/243 (74%)

Train Epoch: 5 [0/666 (0%)]	Loss: 0.092100
Start testing...

Test set: Average loss: 0.0465, Accuracy: 179/243 (74%)

Train Epoch: 6 [0/666 (0%)]	Loss: 0.056113
Start testing...

Test set: Average loss: 0.0412, Accuracy: 179/243 (74%)

Train Epoch: 7 [0/666 (0%)]	Loss: 0.065719
Start testing...

Test set: Average loss: 0.0342, Accuracy: 179/243 (74%)

Train Epoch: 8 [0/666 (0%)]	Loss: 0.071334
Start testing...

Test set: Average loss: 0.0339, Accuracy: 179/243 (74%)

Train Epoch: 9 [0/666 (0%)]	Loss: 0.042990
Start testing...

Test set: Average loss: 0.0293, Accuracy: 179/243 (74%)

Train Epoch: 10 [0/666 (0%)]	Loss: 0.049357
Start testin

This is the part I load the model, the code above trains the model and saved it. 

In [ ]:
torch.load(F"/content/drive/MyDrive/Data/Model/{model_testing_name}")

Now we can actually train the model

In [ ]:
pip install pytorch2keras 


     |████████████████████████████████| 12.8 MB 5.6 MB/s 
     |████████████████████████████████| 462 kB 45.2 MB/s 
  Created wheel for pytorch2keras: filename=pytorch2keras-0.2.4-py3-none-any.whl size=29677 sha256=edaa3ffa451dcf11bc31a3fb3d858bab780c6c3a7945d5b8ed3091878e49aaa3
  Stored in directory: /root/.cache/pip/wheels/72/4e/ee/8c004883e677ab4283783ffd9433ecf595327889dd367c79b1
  Created wheel for onnx2keras: filename=onnx2keras-0.0.24-py3-none-any.whl size=24593 sha256=87b5ad4531d120441b9cb82c02ba29afb06d7fcd60295c2ed99d2ad607c7b6ec
  Stored in directory: /root/.cache/pip/wheels/21/29/29/227fc9f8fed234b12169ae41f979cfadf1dcdbe1b370a5bbb5
Successfully built pytorch2keras onnx2keras


In [ ]:
def pytorch_to_keras(
    model, args, input_shapes=None,
    change_ordering=False, verbose=False, name_policy=None,
    use_optimizer=False, do_constant_folding=False
):

    # ...

    # load a ModelProto structure with ONNX
    onnx_model = onnx.load(stream)

    # ...
    #
    k_model = onnx_to_keras(onnx_model=onnx_model, input_names=input_names,
                            input_shapes=input_shapes, name_policy=name_policy,
                            verbose=verbose, change_ordering=change_ordering)

    return k_model

In [ ]:
import onnx
import torch

example_input = get_example_input() # exmample for the forward pass input 
pytorch_model = get_pytorch_model()
ONNX_PATH="./my_model.onnx"

torch.onnx.export(
    model=pytorch_model,
    args=example_input, 
    f=ONNX_PATH, # where should it be saved
    verbose=False,
    export_params=True,
    do_constant_folding=False,  # fold constant values for optimization
    # do_constant_folding=True,   # fold constant values for optimization
    input_names=['input'],
    output_names=['output']
)
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)

NameError: ignored